## Kullback-Leibler Divergence of Empirical and Theoretical Probabilities of Rankings 

The Kullback-Leibler(KL) divergence of two probability distributions is a measure of difference between the two probability distributions. For probability distributions E and T, the KL divergence is 

$$ D_{KL}(P, Q) = \sum_{i}Q(i)\log\frac{Q(i)}{P(i)}    $$

where i is the ith term that the probability distribution is defined over. To find the KL divergence between the empirical and theoretical probability distributions of the Ireland 2002 data, we first load in the data as well as the parameters we found for the Mallows and Plackett-Luce models that best fit the data:

In [1]:
import readPreflib
import numpy as np

_, lengths, votes = readPreflib.soiInputwithWeights('data_input/ED-debian-2002.soi')
num_votes = 1.0 * sum(lengths.values())

import pickle

mallows_params  = pickle.load( open('pickle/mallows2002_1mil_2.p','rb') )
sigma, phi = mallows_params
plackett_params = pickle.load( open('pickle/plackett2002_3mil_2.p','rb')) 
pl_weights = plackett_params

mallows_params, plackett_params, num_votes

([array([3, 1, 5, 4, 2]), 0.006131716274196619],
 array([0.31115367, 0.24348515, 0.33875853, 0.10660266]),
 475.0)

We also need to gather the probability functions for the Mallows and Plackett-Luce models

In [13]:
import import_ipynb
from Mallows_Notebook import *
from PL_Notebook import *
import metropolis
import math
from tqdm import tqdm_notebook
import itertools

Now we can follow the equation for KL divergence to find it.

In [3]:
divergence_mallows = 0
divergence_plackett = 0

def insideSum(Qi,Pi):
    return Qi * math.log(Qi/Pi)

for entry in votes:
    num_occurances, vote = entry
    empirical = num_occurances / num_votes
    mallows = mallowsProb(vote, sigma, phi)
    plackett = probPlackett(vote, pl_weights)
    divergence_mallows += insideSum(mallows, empirical)
    divergence_plackett += insideSum(plackett, empirical)

results = [divergence_mallows, divergence_plackett]
results

[372.9318955776771, 46.746431392831106]

Save results

In [4]:
import pickle

pickle.dump(results, open('pickle/divergence_.p','wb'))

This can be done for all the files in a folder as well

In [17]:
files = ['ED-00002-00000001.soi',\
         'ED-00002-00000002.soi',\
         'ED-00002-00000003.soi',\
         'ED-00002-00000004.soi',\
         'ED-00002-00000005.soi',\
         'ED-00002-00000006.soi',\
         'ED-00002-00000007.soi']


list_of_votes = []
mallows_params = []
pl_params = []

nruns = 1_000

print('Projected Time =',120/1000.0 * nruns,'seconds, which is ~', 2/1000.0 * nruns, 'minutes')

for file in tqdm_notebook(files,desc = 'All Files'):
    _, lengths, votes = readPreflib.soiInputwithWeights('preflib_soi/'+file)
    num_votes = 1.0 * sum(lengths.values())
    list_of_votes.append((num_votes, lengths, votes))
    p_mal = runMallows(votes, nruns, lengths)
    mallows_params.append(p_mal)
    p_pl = runPL(votes, nruns, lengths)
    pl_params.append(p_pl)

Projected Time = 120.0 seconds, which is ~ 2.0 minutes


In [12]:
import pickle

pickle.dump([mallows_params, pl_params], open('pickle/params_1k.p','wb'))

So here is what we would do if we wanted to iterate over all the unique votes in the dataset, but we dont want to do that :^)

In [18]:
table = []
col_names = ['Number_Votes','Mallow\'s_Divergence','Plackett-Luce_Divergence']

for i in range(len(list_of_votes)):
    num_votes, lengths, votes = list_of_votes[i]
    sigma, phi = mallows_params[i]
    pl_weights = pl_params[i]
    
    divergence_mallows = 0
    divergence_plackett = 0
    
    for entry in votes:
        num_occurances, vote = entry
        empirical = num_occurances / num_votes
        mallows = mallowsProb(vote, sigma, phi)
        plackett = probPlackett(vote, pl_weights)
        divergence_mallows += insideSum(mallows, empirical)
        divergence_plackett += insideSum(plackett, empirical)
    
    table.append([num_votes, divergence_mallows, divergence_plackett])

But instead we want to iterate over all possible votes. We do that like this:

No we dont, KL divergence doesnt work with 0s

In [15]:
table = []
col_names = ['Number_Votes','Mallow\'s_Divergence','Plackett-Luce_Divergence']


for i in range(len(list_of_votes)):
    num_votes, lengths, votes = list_of_votes[i]
    sigma, phi = mallows_params[i]
    pl_weights = pl_params[i]
    
    divergence_mallows = 0
    divergence_plackett = 0
    
    possibles = len(range(len(lengths)))
    pass
    
#     num_votes, votes = list_of_votes[i]
#     sigma, phi = mallows_params[i]
#     pl_weights = pl_params[i]
    
#     divergence_mallows = 0
#     divergence_plackett = 0
    
#     for entry in votes:
#         num_occurances, vote = entry
#         empirical = num_occurances / num_votes
#         mallows = mallowsProb(vote, sigma, phi)
#         plackett = probPlackett(vote, pl_weights)
#         divergence_mallows += insideSum(mallows, empirical)
#         divergence_plackett += insideSum(plackett, empirical)
    
#     table.append([num_votes, divergence_mallows, divergence_plackett])

In [1]:
# possibles = list(range(len(lengths))) + 1
possibles, lengths

NameError: name 'possibles' is not defined

In [19]:
import pandas as pd

npdata = np.array(table)
results_df = pd.DataFrame(data=npdata,columns=col_names)
results_df.index += 1
pickle.dump(results_df, open('pickle/divergence10k.p','wb'))
results_df

,Number_Votes,Mallow's_Divergence,Plackett-Luce_Divergence
1,475.0,340.338764,44.709738
2,488.0,449.728071,61.815098
3,504.0,606.433668,92.739668
4,421.0,3.267751,76.153729
5,482.0,12.643678,77.891219
6,436.0,302.960155,60.120151
7,403.0,364.444871,39.241049


In [8]:
# pickle.load(open('pickle/divergence100k.p','rb'))

In [9]:
results_df.to_csv('pickle/10kRunsDivergence.csv',sep=',')